In [2]:
import utils

model = {"P":False, "Q":True, "L":True, "R":True}

alpha = "(P|Q) | F"
a = utils.expr(alpha)
print(a.args[0].op)

|


In [ ]:
def PL_True(expr, model):
    # Check if the expression is a single variable or a constant
    if expr.op is None:
        # Return the boolean value from the model dictionary
        return model[expr.name]

    # Handle the logical OR operator
    elif expr.op == '|':
        # Recursively evaluate each argument and return True if any are True
        return PL_True(expr.args[0], model) or PL_True(expr.args[1], model)

    # Handle the logical AND operator
    elif expr.op == '&':
        # Recursively evaluate each argument and return True if both are True
        return PL_True(expr.args[0], model) and PL_True(expr.args[1], model)

    # Handle the logical NOT operator
    elif expr.op == '~':
        # Recursively evaluate the argument and return the negation
        return not PL_True(expr.args[0], model)
# if an unsupported op is passed 


# Example usage
model = {'P': False, 'Q': True, 'L': True, 'R': True, 'F': False}
alpha = "(P|Q) | F"

# Parse the expression using a helper function from the utils module
parsed_expr = utils.expr(alpha)
result = PL_True(parsed_expr, model)
print(result)  # Should output the result of the logical expression evaluation


None


In [ ]:
def eval_expr(expression, model):
    if expression in model:
        # Base case if the expression is a single variable or a constant: Return the boolean value directly from the model
        return model[expression]

    # Handle the NOT operation 
    if expression.startswith('~'):
        # Recursively evaluate the NOT operand (descending into a subtree)and return the logical negation of the result
        return not eval_expr(expression[1:], model)

    # Initialize variables to find the main operator at the outermost level
    depth = 0  # Depth tracks the level of nested parentheses
    main_op = None  # To store the main operator outside any parentheses

    # Loop through each character in the expression to find the main operator
    for i, char in enumerate(expression):
        if char == '(':
            depth += 1
        elif char == ')':
            depth -= 1
        # Check for operators only at the outermost level (depth == 0)
        elif depth == 0 and (char == '|' or char == '&'):
            main_op = char
            op_index = i
            break

    # If a main operator is found, process it as a binary node
    if main_op:
        # Extract left and right sub-expressions surrounding the main operator
        left = expression[:op_index].strip()
        right = expression[op_index+1:].strip()
        
        # Recursively evaluate each sub-expression (descending into left and right subtrees)
        # For OR operator, return True if either recursive call returns True
        if main_op == '|':
            return eval_expr(left, model) or eval_expr(right, model)
        # For AND operator, return True only if both recursive calls return True
        elif main_op == '&':
            return eval_expr(left, model) and eval_expr(right, model)

    # Handle expressions surrounded by parentheses (like handling a nested subtree)
    if expression.startswith('(') and expression.endswith(')'):
        # Strip the outermost parentheses and recursively evaluate the inner expression
        return eval_expr(expression[1:-1].strip(), model)

    # If none of the above conditions are met, raise an error for unsupported format
    raise ValueError(f"Unsupported expression format: {expression}")

# Example 
model = {'P': False, 'Q': True, 'L': True, 'R': True, 'F': False}
expression = "((P|Q) | F)"
result = eval_expr(expression, model)
print(result)  # Outputs the result of the logical expression evaluation


True
